In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator
import subprocess
import os
# python version: 3.11.9

In [2]:
# presets, functions

# preset values for the analysis:
#____________________________________________________________________________________________________________________
# physical constants
Z = 20
A = 40
mass_nucleon = 0.938273 
# mass_nucleus = 11.178
mass_nucleus = A * 0.931494
alpha_fine = 1 / 137

# three-momentum bin centers
qvcenters = [0.100, 0.148, 0.167, 0.205, 0.240, 0.300, 0.380, 0.475, 0.570, 0.649, 0.756, 0.991, 1.619, 1.921, 2.213, 2.500, 2.783, 3.500]
# three-momentum edges
qvbins = [0.063, 0.124, 0.158, 0.186, 0.223, 0.270, 0.340, 0.428, 0.523, 0.609, 0.702, 0.878, 1.302, 1.770, 2.067, 2.357, 2.642, 2.923, 4.500]
# four-momentum squared bin names, in string format
qvbin_names = ['[0.063,0.124]', '[0.124,0.158]', '[0.158,0.186]', '[0.186,0.223]', '[0.223,0.270]', '[0.270,0.340]', '[0.340,0.428]', '[0.428,0.523]', '[0.523,0.609]',
                '[0.609,0.702]', '[0.702,0.878]', '[0.878,1.302]', '[1.302,1.770]', '[1.770,2.067]', '[2.067,2.357]', '[2.357,2.642]', '[2.642,2.923]', '[2.923,4.500]']

# four-momentum squared bin centers
Q2centers = [0.010, 0.020, 0.026, 0.040, 0.056, 0.093, 0.120, 0.160, 0.265, 0.380, 0.500, 0.800, 1.250, 1.750, 2.250, 2.750, 3.250, 3.750]
# four-momentum squared bin edges
Q2bins = [0.004, 0.015, 0.025, 0.035, 0.045, 0.070, 0.100, 0.145, 0.206, 0.322, 0.438, 0.650, 1.050, 1.500, 2.000, 2.500, 3.000, 3.500, 4.000]
# four-momentum squared bin names, in string format
Q2bin_names = ['[0.004,0.015]', '[0.015,0.025]', '[0.025,0.035]', '[0.035,0.045]', '[0.045,0.070]', '[0.070,0.100]', '[0.100,0.145]', '[0.145,0.206]', '[0.206,0.322]',
                '[0.322,0.438]', '[0.438,0.650]', '[0.650,1.050]', '[1.050,1.500]', '[1.500,2.000]', '[2.000,2.500]', '[2.500,3.000]', '[3.000,3.500]', '[3.500,4.000]']

dataSet_to_name = {1:"Meziani0.120_90", 2:"Meziani0.120_140", 3:"Williamson0.130_140", 4:"Torizuka0.15_35", 5:"Williamson0.150_90", 6:"Meziani0.160_60", 7:"Meziani0.160_90", 8:"Meziani0.160_140", 9:"Williamson0.160_140 ", 10:"Torizuka0.183_35",
                   11:"Torizuka0.183_45", 12:"Torizuka0.183_55", 13:"Williamson0.189_140", 14:"Mezian0.2_60", 15:"Mezini0.2_90", 16:"Williamson0.2_90", 17:"Meziani0.2_140", 18:"Wlliamson_0.219_140", 19:" Meziani0.24_60", 20:"Meziani0.24_90",
                   21:"Meziani0.24_140", 22:"Williamson0.248_90", 23:"Williamson0.249_140", 24:"Torizuka0.25_50", 25:"Torizuka0.25_60", 26:"Torizuka0.25_70", 27:"Meziani0.28_60", 28:"Willaimson0.28_90", 29:"Meziani0.28_140", 30:"Williamson0.287_140",
                   31:"Williamson0.298_90", 32:"Meziani0.32_90", 33:"Meziani0.32_140", 34:"Williamson0.327_140", 35:" Meziani0.34_60", 36:"Williamson0.347_90", 37:" Williamson0.348_45.5", 38:" Meziani0.36_90", 39:"Meziani0.36_140", 40:"Williamson0.367_140",
                   41:"Williamson0.372_90", 42:"Meziani0.4_60", 43:"Meziani0.4_90", 44:"Meziani0.4_140", 45:"Williamson0.408_45.5", 46:"Meziani0.44_140", 47:" Williamson0.471_45.5", 48:"Meziani0.48_60", 49:"Meziani0.48_140", 50:"Whitney0.5_60",
                   51:"Williamson0.545_45.5", 52:"Meziani0.56_60", 53:"Williamson0.628_45.4", 54:"Williamson0.681_45.5", 55:"Williamson0.739_45.5", 56:"Williamson0.782_45.5", 57:"Williamson0.841_45.5"}

dataSet_to_normalization ={1:1.094, 2:1.080, 3:1.0, 4:1.051 , 5:1.0 , 6:0.964 , 7:0.934 ,8:0.989 , 9:1.0, 10:1.051, 11:1.051 ,
                           12:1.051 , 13:1.0, 14:0.964 , 15:0.970 , 16:1.0, 17:0.937 , 18:1.0, 19:0.964 , 20:0.870 , 21:0.899 ,
                           22:1.0 , 23:1.0 , 24:1.051 , 25:1.051 , 26:1.051 , 27:0.964 , 28:1.0 , 29:0.913 , 30:1.0 , 31:1.0 ,
                           32:1.036 , 33:0.939 , 34:1.0 , 35:0.964 , 36:1.0 , 37:1.0 , 38:0.959 , 39:0.976 , 40:0.976 , 41:1.0 ,
                           42:0.964 , 43:0.927 , 44:0.983 , 45:1.0 , 46:0.970 , 47:1.0 , 48:0.964 , 49:1.001 , 50:0.954 , 51:1.0 ,
                           52:1.0 , 53:1.0 , 54:1.0 , 55:1.0 , 56:1.0 , 57:1.0 }

dataSet_to_normError = {1:0.037, 2:0.037, 3:0.037, 4:0.037 , 5:0.037*2 , 6:0.037 , 7:0.037 ,8:0.037 , 9:0.037, 10:0.037, 11:0.037 ,
                        12:0.037 , 13:0.037, 14:0.037 , 15:0.037 , 16:0.037*2, 17:0.037 , 18:0.037, 19:0.037 , 20:0.037 , 21:0.037 ,
                        22:0.037*2 , 23:0.037 , 24:0.037 , 25:0.037 , 26:0.037 , 27:0.037 , 28:0.037 , 29:0.037 , 30:0.037 , 31:0.037*2 ,
                        32:0.037 , 33:0.037 , 34:0.037 , 35:0.037 , 36:0.037*2 , 37:0.037 , 38:0.037 , 39:0.037 , 40:0.037 , 41:0.037*2 ,
                        42:0.037 , 43:0.037 , 44:0.037 , 45:0.037 , 46:0.037 , 47:0.037 , 48:0.037 , 49:0.037 , 50:0.037 , 51:0.037 ,
                        52:0.037 , 53:0.037 , 54:0.037 , 55:0.037 , 56:0.037 , 57:0.037}

def round_sig_3(x):
    return '%s' % float('%.3g' % x)

def append_row(df, row):
    return pd.concat([df, pd.DataFrame([row], columns=row.index)]).reset_index(drop=True)

def linear_model(x, a, b):
    return a * x + b

# read dataframe from csv file
def prepare_df(filepath):
    # read the csv file:
    df = pd.read_csv(filepath)

    df["Veff"]=0.0074
    
    df["Eeff"] = df["E0"] + df["Veff"]
    df["Ep"] = df["E0"] - df["nu"]
    df["Ep_eff"] = df["Ep"] + df["Veff"]

    # calculate normalized cross section:
    df["normalization"] = df["dataSet"].map(dataSet_to_normalization)
    df["normError"] = df["dataSet"].map(dataSet_to_normError)
    df['normCross'] = df['cross'] * df['normalization']
    df['normCrossError'] = df['normCross'] * np.sqrt((df['error'] / df['cross'])**2 + (df['normError'] / df['normalization'])**2)
    print(df.loc[df['normalization'] == 1, 'dataSet'].unique())
    
    # calculate the kinematic variables:
    df["ThetaRad"] = df["ThetaDeg"]*np.pi/180
    df["sin2(T/2)"] = (np.sin(df["ThetaRad"] / 2))**2
    df["cos2(T/2)"] = (np.cos(df["ThetaRad"] / 2))**2
    df["tan2(T/2)"] = (np.tan(df["ThetaRad"] / 2))**2

    df["nuel"] = df["E0"] - df["E0"] / (1 + 2 * df["E0"] * df["sin2(T/2)"] / mass_nucleus)
    df["Ex"] = df["nu"] - (df["E0"] - df["E0"] / (1 + 2 * df["E0"] * df["sin2(T/2)"] / mass_nucleus))

    df["R"] = 1.1 * (df["A"])**(1/3) + 0.86 / ((df["A"])**(1/3))

    df["F2foc"] = (df["Eeff"] / df["E0"])**2
    df["Q2"] = 4 * df["E0"] * (df["Ep"]) * df["sin2(T/2)"]
    df["Q2eff"] = 4 * df["Eeff"] * df["Ep_eff"] * df["sin2(T/2)"]
    df["qv2"] = df["nu"]**2 + df["Q2eff"]
    df["qv"] = np.sqrt(df["qv2"])

    df["W2"] = mass_nucleon**2 + 2 * mass_nucleon * df["nu"] - df["Q2eff"]
    df["epsilon"] = 1 / (1 + 2 * (1 + (df["nu"]**2) / df["Q2eff"]) * df["tan2(T/2)"])

    df["gamma"] = alpha_fine * df["Ep_eff"] * (df["W2"] - mass_nucleon**2) / (( 4 * ((np.pi)**2) * df["Q2eff"] * mass_nucleon * df["E0"]) * (1 - df["epsilon"]))
    df["Sig_R"] = df["normCross"] / df["gamma"]
    df["D_sig_R"] = df["error"] / df["gamma"]
    df["Sig_mott"] = 4 * (alpha_fine**2) * (df["Ep"]**2) * df["cos2(T/2)"] / (df["Q2"]**2)
    df["Sig_mott_eff"] = df["Sig_mott"] * df["E0"] / df["Eeff"]

    # Calculate the Rosenbluth quantity:
    df["H"] = (df["qv2"]**2) / (4 * (alpha_fine**2) * (df["Ep_eff"]**2) * (df["cos2(T/2)"] + 2 * (df["qv2"] / df["Q2eff"]) * df["sin2(T/2)"]))
    df["Hcc"] = df["H"] / df["F2foc"]
    df["Hstar_Sig(nb)"] = df["H"] * df["normCross"]
    df["Hstar_error(nb)"] = df["H"] * df["normCrossError"]
    df["Hstar_Sig(GeV)"] = df["Hstar_Sig(nb)"] / ((0.1973269**2) * 10000000)
    df["Hstar_error(GeV)"] = df["Hstar_error(nb)"] / ((0.1973269**2) * 10000000)
    df["Hcc_Sig(nb)"] = df["Hcc"] * df["normCross"]
    df["Hcc_error(nb)"] = df["Hcc"] * df["normCrossError"]
    df["Hcc_Sig(GeV)"] = df["Hcc_Sig(nb)"] / ((0.1973269**2) * 10000000)
    df["Hcc_error(GeV)"] = df["Hcc_error(nb)"] / ((0.1973269**2) * 10000000)
    # we will fit "Hcc_Sig(GeV)" vs "epsilon" to the linear model to get the Rosenbluth slope and intercept

    # subdivide the data into bins
    df['qvbin'] = 0
    df['qvcenter'] = 0
    df['Q2bin'] = 0
    df['Q2center'] = 0
    df["qvbin"] = pd.cut(x=df["qv"], bins=qvbins, labels=qvbin_names, right=True)
    df["qvcenter"] = pd.cut(x=df["qv"], bins=qvbins, labels=qvcenters, right=True)
    df['qvcenter'] = pd.to_numeric(df['qvcenter'])
    df["Q2bin"] = pd.cut(x=df["Q2eff"], bins=Q2bins, labels=Q2bin_names, right=True)
    df["Q2center"] = pd.cut(x=df["Q2eff"], bins=Q2bins, labels=Q2centers, right=True)
    df['Q2center'] = pd.to_numeric(df['Q2center'])
    df = df.dropna()
    # now every row of data has a bin (Q2/qv) label

    # this is for bc_qv_ex and bc_q2_ex
    df['ExA'] = np.where(df['Ex'] >= 0.04, df['Ex'], 0.04)
    df['nuA'] = np.where(df['Ex'] >= 0.04, df['nu'], df['nu'] + 0.04 - df['Ex'])
    df["Q2effA"] = 4 * df["Eeff"] * (df["Eeff"] - df['nuA']) * df["sin2(T/2)"]
    df["qv2A"] = df["nuA"]**2 + df["Q2effA"]
    df["epsilonA"] = 1 / (1 + 2 * (1 + (df["nuA"]**2) / df["Q2effA"]) * df["tan2(T/2)"])


    return df

In [3]:
# read csv file
df = prepare_df('Data/Ca40.csv')
df

[ 3  5  9 13 16 18 22 23 28 30 31 34 36 37 41 45 47 51 52 53 54 55 56 57]


,Z,A,E0,ThetaDeg,nu,cross,error,dataSet,normalization,normError,...,Ep,Ep_eff,nuel,qv2,qv,ExA,nuA,Q2effA,qv2A,epsilonA
0,20,40,0.120,90.0,0.007500,216000.00000,18000.000000,1,1.094,0.037,...,0.112500,0.119900,0.000385,0.030607,0.174948,0.040000,0.040385,0.022171,0.023802,0.317750
1,20,40,0.120,90.0,0.012500,92100.00000,9395.856800,1,1.094,0.037,...,0.107500,0.114900,0.000385,0.029433,0.171560,0.040000,0.040385,0.022171,0.023802,0.317750
2,20,40,0.120,90.0,0.017500,138000.00000,12480.785230,1,1.094,0.037,...,0.102500,0.109900,0.000385,0.028309,0.168252,0.040000,0.040385,0.022171,0.023802,0.317750
3,20,40,0.120,90.0,0.022500,124000.00000,12107.848690,1,1.094,0.037,...,0.097500,0.104900,0.000385,0.027235,0.165030,0.040000,0.040385,0.022171,0.023802,0.317750
4,20,40,0.120,90.0,0.027500,92300.00000,10543.653300,1,1.094,0.037,...,0.092500,0.099900,0.000385,0.026211,0.161897,0.040000,0.040385,0.022171,0.023802,0.317750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3817,20,40,0.841,45.5,0.505880,11255.64210,562.782105,57,1.000,0.037,...,0.335120,0.342520,0.005639,0.429743,0.655548,0.500241,0.505880,0.173828,0.429743,0.534919
3818,20,40,0.841,45.5,0.510891,11188.65472,559.432736,57,1.000,0.037,...,0.330109,0.337509,0.005639,0.432295,0.657491,0.505252,0.510891,0.171285,0.432295,0.529776
3819,20,40,0.841,45.5,0.515902,11107.46445,555.373222,57,1.000,0.037,...,0.325098,0.332498,0.005639,0.434897,0.659467,0.510263,0.515902,0.168742,0.434897,0.524551
3820,20,40,0.841,45.5,0.520914,11011.38301,550.569151,57,1.000,0.037,...,0.320086,0.327486,0.005639,0.437549,0.661475,0.515274,0.520914,0.166198,0.437549,0.519245


In [4]:
response_columns = ['i','RTTOT','RLTOT','RTQE','RLQE','RTNS','RLNS']
C12_factor = Z / 6
# 
QENS_shift = 0.013
NS_factor = 0.5
df = df[df['nu'] > QENS_shift].reset_index(drop=True)
df_shift = df.copy()
df_shift['nu'] = df['nu'] - QENS_shift
#

# calculate bc_qv_nu
df[['qvcenter','nu']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_qv_nu.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
#
df_shift[['qvcenter','nu']].to_csv('input_shift.txt',index=True,header=False,sep=' ')
with open('output_shift.txt', 'w') as output_file:
    subprocess.run(['./response_qv_nu.exe', 'input_shift.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response_shift = pd.read_csv('output_shift.txt', sep='\s+', header=None, names=response_columns)
df_response['RTTOT'] = df_response['RTTOT'] - df_response['RTQE'] + df_response_shift['RTQE'] - df_response['RTNS'] + df_response_shift['RTNS'] * NS_factor
df_response['RLTOT'] = df_response['RLTOT'] - df_response['RLQE'] + df_response_shift['RLQE'] - df_response['RLNS'] + df_response_shift['RLNS'] * NS_factor
df_response['RTQE'] = df_response_shift['RTQE']
df_response['RLQE'] = df_response_shift['RLQE']
df_response['RTNS'] = df_response_shift['RTNS'] * NS_factor
df_response['RLNS'] = df_response_shift['RLNS'] * NS_factor
#
df['RL_qvc_nu'] = df.index.map(df_response.set_index('i')['RLTOT']) # qvc: qv at bin center
df['RT_qvc_nu'] = df.index.map(df_response.set_index('i')['RTTOT']) # qvd: qv of data

df[['qv','nu']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_qv_nu.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
#
df_shift[['qv','nu']].to_csv('input_shift.txt',index=True,header=False,sep=' ')
with open('output_shift.txt', 'w') as output_file:
    subprocess.run(['./response_qv_nu.exe', 'input_shift.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response_shift = pd.read_csv('output_shift.txt', sep='\s+', header=None, names=response_columns)
df_response['RTTOT'] = df_response['RTTOT'] - df_response['RTQE'] + df_response_shift['RTQE'] - df_response['RTNS'] + df_response_shift['RTNS'] * NS_factor
df_response['RLTOT'] = df_response['RLTOT'] - df_response['RLQE'] + df_response_shift['RLQE'] - df_response['RLNS'] + df_response_shift['RLNS'] * NS_factor
df_response['RTQE'] = df_response_shift['RTQE']
df_response['RLQE'] = df_response_shift['RLQE']
df_response['RTNS'] = df_response_shift['RTNS'] * NS_factor
df_response['RLNS'] = df_response_shift['RLNS'] * NS_factor
#
df['RL_qvd_nu'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_qvd_nu'] = df.index.map(df_response.set_index('i')['RTTOT'])
print('RL RT qv_nu done.')

# calculate bc_qv_ex
df[['qvcenter','ExA']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_qv_ex.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
#
df_shift[['qvcenter','ExA']].to_csv('input_shift.txt',index=True,header=False,sep=' ')
with open('output_shift.txt', 'w') as output_file:
    subprocess.run(['./response_qv_ex.exe', 'input_shift.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response_shift = pd.read_csv('output_shift.txt', sep='\s+', header=None, names=response_columns)
df_response['RTTOT'] = df_response['RTTOT'] - df_response['RTQE'] + df_response_shift['RTQE'] - df_response['RTNS'] + df_response_shift['RTNS'] * NS_factor
df_response['RLTOT'] = df_response['RLTOT'] - df_response['RLQE'] + df_response_shift['RLQE'] - df_response['RLNS'] + df_response_shift['RLNS'] * NS_factor
df_response['RTQE'] = df_response_shift['RTQE']
df_response['RLQE'] = df_response_shift['RLQE']
df_response['RTNS'] = df_response_shift['RTNS'] * NS_factor
df_response['RLNS'] = df_response_shift['RLNS'] * NS_factor
#
df['RL_qvc_ex'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_qvc_ex'] = df.index.map(df_response.set_index('i')['RTTOT'])

df[['qv','ExA']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_qv_ex.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
#
df_shift[['qv','ExA']].to_csv('input_shift.txt',index=True,header=False,sep=' ')
with open('output_shift.txt', 'w') as output_file:
    subprocess.run(['./response_qv_ex.exe', 'input_shift.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response_shift = pd.read_csv('output_shift.txt', sep='\s+', header=None, names=response_columns)
df_response['RTTOT'] = df_response['RTTOT'] - df_response['RTQE'] + df_response_shift['RTQE'] - df_response['RTNS'] + df_response_shift['RTNS'] * NS_factor
df_response['RLTOT'] = df_response['RLTOT'] - df_response['RLQE'] + df_response_shift['RLQE'] - df_response['RLNS'] + df_response_shift['RLNS'] * NS_factor
df_response['RTQE'] = df_response_shift['RTQE']
df_response['RLQE'] = df_response_shift['RLQE']
df_response['RTNS'] = df_response_shift['RTNS'] * NS_factor
df_response['RLNS'] = df_response_shift['RLNS'] * NS_factor
#
df['RL_qvd_ex'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_qvd_ex'] = df.index.map(df_response.set_index('i')['RTTOT'])
print('RL RT qv_ex done.')

# calculate bc_qv_w2
df[['qvcenter','W2']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_qv_w2.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
#
df_shift[['qvcenter','W2']].to_csv('input_shift.txt',index=True,header=False,sep=' ')
with open('output_shift.txt', 'w') as output_file:
    subprocess.run(['./response_qv_w2.exe', 'input_shift.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response_shift = pd.read_csv('output_shift.txt', sep='\s+', header=None, names=response_columns)
df_response['RTTOT'] = df_response['RTTOT'] - df_response['RTQE'] + df_response_shift['RTQE'] - df_response['RTNS'] + df_response_shift['RTNS'] * NS_factor
df_response['RLTOT'] = df_response['RLTOT'] - df_response['RLQE'] + df_response_shift['RLQE'] - df_response['RLNS'] + df_response_shift['RLNS'] * NS_factor
df_response['RTQE'] = df_response_shift['RTQE']
df_response['RLQE'] = df_response_shift['RLQE']
df_response['RTNS'] = df_response_shift['RTNS'] * NS_factor
df_response['RLNS'] = df_response_shift['RLNS'] * NS_factor
#
df['RL_qvc_w2'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_qvc_w2'] = df.index.map(df_response.set_index('i')['RTTOT'])

df[['qv','W2']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_qv_w2.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
#
df_shift[['qv','W2']].to_csv('input_shift.txt',index=True,header=False,sep=' ')
with open('output_shift.txt', 'w') as output_file:
    subprocess.run(['./response_qv_w2.exe', 'input_shift.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response_shift = pd.read_csv('output_shift.txt', sep='\s+', header=None, names=response_columns)
df_response['RTTOT'] = df_response['RTTOT'] - df_response['RTQE'] + df_response_shift['RTQE'] - df_response['RTNS'] + df_response_shift['RTNS'] * NS_factor
df_response['RLTOT'] = df_response['RLTOT'] - df_response['RLQE'] + df_response_shift['RLQE'] - df_response['RLNS'] + df_response_shift['RLNS'] * NS_factor
df_response['RTQE'] = df_response_shift['RTQE']
df_response['RLQE'] = df_response_shift['RLQE']
df_response['RTNS'] = df_response_shift['RTNS'] * NS_factor
df_response['RLNS'] = df_response_shift['RLNS'] * NS_factor
#
df['RL_qvd_w2'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_qvd_w2'] = df.index.map(df_response.set_index('i')['RTTOT'])
print('RL RT qv_w2 done.')

# calculate bc_q2_nu
df[['Q2center','nu']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_q2_nu.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
#
df_shift[['Q2center','nu']].to_csv('input_shift.txt',index=True,header=False,sep=' ')
with open('output_shift.txt', 'w') as output_file:
    subprocess.run(['./response_q2_nu.exe', 'input_shift.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response_shift = pd.read_csv('output_shift.txt', sep='\s+', header=None, names=response_columns)
df_response['RTTOT'] = df_response['RTTOT'] - df_response['RTQE'] + df_response_shift['RTQE'] - df_response['RTNS'] + df_response_shift['RTNS'] * NS_factor
df_response['RLTOT'] = df_response['RLTOT'] - df_response['RLQE'] + df_response_shift['RLQE'] - df_response['RLNS'] + df_response_shift['RLNS'] * NS_factor
df_response['RTQE'] = df_response_shift['RTQE']
df_response['RLQE'] = df_response_shift['RLQE']
df_response['RTNS'] = df_response_shift['RTNS'] * NS_factor
df_response['RLNS'] = df_response_shift['RLNS'] * NS_factor
#
df['RL_q2c_nu'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_q2c_nu'] = df.index.map(df_response.set_index('i')['RTTOT'])

df[['Q2eff','nu']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_q2_nu.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
#
df_shift[['Q2eff','nu']].to_csv('input_shift.txt',index=True,header=False,sep=' ')
with open('output_shift.txt', 'w') as output_file:
    subprocess.run(['./response_q2_nu.exe', 'input_shift.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response_shift = pd.read_csv('output_shift.txt', sep='\s+', header=None, names=response_columns)
df_response['RTTOT'] = df_response['RTTOT'] - df_response['RTQE'] + df_response_shift['RTQE'] - df_response['RTNS'] + df_response_shift['RTNS'] * NS_factor
df_response['RLTOT'] = df_response['RLTOT'] - df_response['RLQE'] + df_response_shift['RLQE'] - df_response['RLNS'] + df_response_shift['RLNS'] * NS_factor
df_response['RTQE'] = df_response_shift['RTQE']
df_response['RLQE'] = df_response_shift['RLQE']
df_response['RTNS'] = df_response_shift['RTNS'] * NS_factor
df_response['RLNS'] = df_response_shift['RLNS'] * NS_factor
#
df['RL_q2d_nu'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_q2d_nu'] = df.index.map(df_response.set_index('i')['RTTOT'])
print('RL RT q2_nu done.')

# calculate bc_q2_ex
df[['Q2center','ExA']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_q2_ex.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
#
df_shift[['Q2center','ExA']].to_csv('input_shift.txt',index=True,header=False,sep=' ')
with open('output_shift.txt', 'w') as output_file:
    subprocess.run(['./response_q2_ex.exe', 'input_shift.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response_shift = pd.read_csv('output_shift.txt', sep='\s+', header=None, names=response_columns)
df_response['RTTOT'] = df_response['RTTOT'] - df_response['RTQE'] + df_response_shift['RTQE'] - df_response['RTNS'] + df_response_shift['RTNS'] * NS_factor
df_response['RLTOT'] = df_response['RLTOT'] - df_response['RLQE'] + df_response_shift['RLQE'] - df_response['RLNS'] + df_response_shift['RLNS'] * NS_factor
df_response['RTQE'] = df_response_shift['RTQE']
df_response['RLQE'] = df_response_shift['RLQE']
df_response['RTNS'] = df_response_shift['RTNS'] * NS_factor
df_response['RLNS'] = df_response_shift['RLNS'] * NS_factor
#
df['RL_q2c_ex'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_q2c_ex'] = df.index.map(df_response.set_index('i')['RTTOT'])

df[['Q2eff','ExA']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_q2_ex.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
#
df_shift[['Q2eff','ExA']].to_csv('input_shift.txt',index=True,header=False,sep=' ')
with open('output_shift.txt', 'w') as output_file:
    subprocess.run(['./response_q2_ex.exe', 'input_shift.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response_shift = pd.read_csv('output_shift.txt', sep='\s+', header=None, names=response_columns)
df_response['RTTOT'] = df_response['RTTOT'] - df_response['RTQE'] + df_response_shift['RTQE'] - df_response['RTNS'] + df_response_shift['RTNS'] * NS_factor
df_response['RLTOT'] = df_response['RLTOT'] - df_response['RLQE'] + df_response_shift['RLQE'] - df_response['RLNS'] + df_response_shift['RLNS'] * NS_factor
df_response['RTQE'] = df_response_shift['RTQE']
df_response['RLQE'] = df_response_shift['RLQE']
df_response['RTNS'] = df_response_shift['RTNS'] * NS_factor
df_response['RLNS'] = df_response_shift['RLNS'] * NS_factor
#
df['RL_q2d_ex'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_q2d_ex'] = df.index.map(df_response.set_index('i')['RTTOT'])
print('RL RT q2_ex done.')

# calculate bc_q2_w2
df[['Q2center','W2']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_q2_w2.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
#
df_shift[['Q2center','W2']].to_csv('input_shift.txt',index=True,header=False,sep=' ')
with open('output_shift.txt', 'w') as output_file:
    subprocess.run(['./response_q2_w2.exe', 'input_shift.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response_shift = pd.read_csv('output_shift.txt', sep='\s+', header=None, names=response_columns)
df_response['RTTOT'] = df_response['RTTOT'] - df_response['RTQE'] + df_response_shift['RTQE'] - df_response['RTNS'] + df_response_shift['RTNS'] * NS_factor
df_response['RLTOT'] = df_response['RLTOT'] - df_response['RLQE'] + df_response_shift['RLQE'] - df_response['RLNS'] + df_response_shift['RLNS'] * NS_factor
df_response['RTQE'] = df_response_shift['RTQE']
df_response['RLQE'] = df_response_shift['RLQE']
df_response['RTNS'] = df_response_shift['RTNS'] * NS_factor
df_response['RLNS'] = df_response_shift['RLNS'] * NS_factor
#
df['RL_q2c_w2'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_q2c_w2'] = df.index.map(df_response.set_index('i')['RTTOT'])

df[['Q2eff','W2']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_q2_w2.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
#
df_shift[['Q2eff','W2']].to_csv('input_shift.txt',index=True,header=False,sep=' ')
with open('output_shift.txt', 'w') as output_file:
    subprocess.run(['./response_q2_w2.exe', 'input_shift.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response_shift = pd.read_csv('output_shift.txt', sep='\s+', header=None, names=response_columns)
df_response['RTTOT'] = df_response['RTTOT'] - df_response['RTQE'] + df_response_shift['RTQE'] - df_response['RTNS'] + df_response_shift['RTNS'] * NS_factor
df_response['RLTOT'] = df_response['RLTOT'] - df_response['RLQE'] + df_response_shift['RLQE'] - df_response['RLNS'] + df_response_shift['RLNS'] * NS_factor
df_response['RTQE'] = df_response_shift['RTQE']
df_response['RLQE'] = df_response_shift['RLQE']
df_response['RTNS'] = df_response_shift['RTNS'] * NS_factor
df_response['RLNS'] = df_response_shift['RLNS'] * NS_factor
#
df['RL_q2d_w2'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_q2d_w2'] = df.index.map(df_response.set_index('i')['RTTOT'])
print('RL RT q2_w2 done.')


<>:16: SyntaxWarning: invalid escape sequence '\s'
<>:22: SyntaxWarning: invalid escape sequence '\s'
<>:37: SyntaxWarning: invalid escape sequence '\s'
<>:43: SyntaxWarning: invalid escape sequence '\s'
<>:60: SyntaxWarning: invalid escape sequence '\s'
<>:66: SyntaxWarning: invalid escape sequence '\s'
<>:81: SyntaxWarning: invalid escape sequence '\s'
<>:87: SyntaxWarning: invalid escape sequence '\s'
<>:104: SyntaxWarning: invalid escape sequence '\s'
<>:110: SyntaxWarning: invalid escape sequence '\s'
<>:125: SyntaxWarning: invalid escape sequence '\s'
<>:131: SyntaxWarning: invalid escape sequence '\s'
<>:148: SyntaxWarning: invalid escape sequence '\s'
<>:154: SyntaxWarning: invalid escape sequence '\s'
<>:169: SyntaxWarning: invalid escape sequence '\s'
<>:175: SyntaxWarning: invalid escape sequence '\s'
<>:192: SyntaxWarning: invalid escape sequence '\s'
<>:198: SyntaxWarning: invalid escape sequence '\s'
<>:213: SyntaxWarning: invalid escape sequence '\s'
<>:219: SyntaxWarnin

RL RT qv_nu done.
RL RT qv_ex done.
RL RT qv_w2 done.
RL RT q2_nu done.
RL RT q2_ex done.
RL RT q2_w2 done.


In [5]:
# determine bin-centering corrections
df["bc_qv_nu"] = 1.0
df["bc_qv_ex"] = 1.0
df["bc_qv_w2"] = 1.0
df["bc_q2_nu"] = 1.0
df["bc_q2_ex"] = 1.0
df["bc_q2_w2"] = 1.0
for qvcenter in qvcenters:
    picked = df.loc[df["qvcenter"] == qvcenter]
    for index, row in picked.iterrows():
        if row['qvcenter'] != float('NaN') and row['Ex'] >= 0.025:
            try:
                df.loc[index, 'bc_qv_nu'] = (row['epsilon'] * row['RL_qvc_nu'] + 0.5 * ((qvcenter**2) / (qvcenter**2 - row['nu']**2)) * row['RT_qvc_nu']) / (row['epsilon'] * row['RL_qvd_nu'] + 0.5 * (row['qv2'] / row['Q2eff']) * row['RT_qvd_nu'])
            except ZeroDivisionError:
                print(f'Skipping index {index} for bc_qv_nu due to zero denominator.')
            try:
                df.loc[index, 'bc_qv_ex'] = (row['epsilonA'] * row['RL_qvc_ex'] + 0.5 * ((qvcenter**2) / (qvcenter**2 - row['nuA']**2)) * row['RT_qvc_ex']) / (row['epsilonA'] * row['RL_qvd_ex'] + 0.5 * (row['qv2A'] / row['Q2effA']) * row['RT_qvd_ex'])
            except ZeroDivisionError:
                print(f'Skipping index {index} for bc_qv_ex due to zero denominator.')
            try:
                df.loc[index, 'bc_qv_w2'] = (row['epsilon'] * row['RL_qvc_w2'] + 0.5 * ((qvcenter**2) / (2 * mass_nucleon * np.sqrt(qvcenter**2 + row['W2']) - row['W2'] - mass_nucleon**2)) * row['RT_qvc_w2']) / (row['epsilon'] * row['RL_qvd_w2'] + 0.5 * (row['qv2'] / row['Q2eff']) * row['RT_qvd_w2'])
            except ZeroDivisionError:
                print(f'Skipping index {index} for bc_qv_w2 due to zero denominator.')

for Q2center in Q2centers:
    picked = df.loc[df["Q2center"] == Q2center]
    for index, row in picked.iterrows():
        if row['Q2center'] != float('NaN') and row['Ex'] >= 0.025:
            nu = (row['W2'] + Q2center - mass_nucleon**2) / (2 * mass_nucleon)
            qv2center = Q2center + nu**2
            try:
                df.loc[index, 'bc_q2_nu'] = (row['epsilon'] * row['RL_q2c_nu'] + 0.5 * ((Q2center + row["nu"]**2) / (Q2center)) * row['RT_q2c_nu']) / (row['epsilon'] * row['RL_q2d_nu'] + 0.5 * (row['qv2'] / row['Q2eff']) * row['RT_q2d_nu'])
            except ZeroDivisionError:
                print(f'Skipping index {index} for bc_q2_nu due to zero denominator.')
            try:
                df.loc[index, 'bc_q2_ex'] = (row['epsilonA'] * row['RL_q2c_ex'] + 0.5 * ((Q2center + row["nuA"]**2) / (Q2center)) * row['RT_q2c_ex']) / (row['epsilonA'] * row['RL_q2d_ex'] + 0.5 * (row['qv2A'] / row['Q2effA']) * row['RT_q2d_ex'])
            except ZeroDivisionError:
                print(f'Skipping index {index} for bc_q2_ex due to zero denominator.')
            try:
                df.loc[index, 'bc_q2_w2'] = (row['epsilon'] * row['RL_q2c_w2'] + 0.5 * (qv2center / (Q2center)) * row['RT_q2c_w2']) / (row['epsilon'] * row['RL_q2d_w2'] + 0.5 * (row['qv2'] / row['Q2eff']) * row['RT_q2d_w2'])
            except ZeroDivisionError:
                print(f'Skipping index {index} for bc_q2_w2 due to zero denominator.')

In [6]:
# save dataframe to csv
df.to_csv('Data/df_Ca40.csv',index=False)